In [17]:
from src.utils import bcli
import json

In [18]:
# Create a wallet
try:
    bcli("createwallet misfit-core")
except:
    print("misfit-wallet already exists")

misfit-wallet already exists


In [19]:
# Generate new address for wallet
addr = bcli("getnewaddress")
addr

'bcrt1q4hs29pvwyndf9370zfzvws7c8lmtpk3r427ltz'

In [20]:
# Generate funds to the address
blockhash = json.loads(bcli(f"generatetoaddress 101 {addr}"))[0]
blockhash

'3712b08e0420ee37d5b4846525d55ef60b9043652451f0472769ea30a3a71ecf'

In [21]:
# Get block data
block = json.loads(bcli(f"getblock {blockhash} 2"))
block

{'hash': '3712b08e0420ee37d5b4846525d55ef60b9043652451f0472769ea30a3a71ecf',
 'confirmations': 101,
 'height': 252,
 'version': 536870912,
 'versionHex': '20000000',
 'merkleroot': '72e184f19ed7abf61b5c50a3c9a215a515211e603a1b4ed3de911ec71dcaab06',
 'time': 1740180081,
 'mediantime': 1740172025,
 'nonce': 1,
 'bits': '207fffff',
 'difficulty': 4.656542373906925e-10,
 'chainwork': '00000000000000000000000000000000000000000000000000000000000001fa',
 'nTx': 1,
 'previousblockhash': '1aef74cd22c2636e9c53603471acd4a746f3039364dc389c25ca3f9647d6cd0f',
 'nextblockhash': '3740867fca3c8cec8b0c96ccd98eff3c25282fee68fa6c95c5804491bcdea57a',
 'strippedsize': 214,
 'size': 250,
 'weight': 892,
 'tx': [{'txid': '72e184f19ed7abf61b5c50a3c9a215a515211e603a1b4ed3de911ec71dcaab06',
   'hash': 'd93276d4c507fab0eb0b66b6e0a23f5179264311be0144171be4d7fb68920ca0',
   'version': 2,
   'size': 169,
   'vsize': 142,
   'weight': 568,
   'locktime': 0,
   'vin': [{'coinbase': '02fc0000',
     'txinwitness': ['00

In [22]:
# Get transaction from block
tx = block['tx'][0]
tx

{'txid': '72e184f19ed7abf61b5c50a3c9a215a515211e603a1b4ed3de911ec71dcaab06',
 'hash': 'd93276d4c507fab0eb0b66b6e0a23f5179264311be0144171be4d7fb68920ca0',
 'version': 2,
 'size': 169,
 'vsize': 142,
 'weight': 568,
 'locktime': 0,
 'vin': [{'coinbase': '02fc0000',
   'txinwitness': ['0000000000000000000000000000000000000000000000000000000000000000'],
   'sequence': 4294967295}],
 'vout': [{'value': 25.0,
   'n': 0,
   'scriptPubKey': {'asm': '0 ade0a2858e24da92c7cf1244c743d83ff6b0da23',
    'desc': 'addr(bcrt1q4hs29pvwyndf9370zfzvws7c8lmtpk3r427ltz)#edpgjkqt',
    'hex': '0014ade0a2858e24da92c7cf1244c743d83ff6b0da23',
    'address': 'bcrt1q4hs29pvwyndf9370zfzvws7c8lmtpk3r427ltz',
    'type': 'witness_v0_keyhash'}},
  {'value': 0.0,
   'n': 1,
   'scriptPubKey': {'asm': 'OP_RETURN aa21a9ede2f61c3f71d1defd3fa999dfa36953755c690689799962b48bebd836974e8cf9',
    'desc': 'raw(6a24aa21a9ede2f61c3f71d1defd3fa999dfa36953755c690689799962b48bebd836974e8cf9)#cav96mf3',
    'hex': '6a24aa21a9ede2f61

In [23]:
# Create valid raw transaction
tx_in = json.dumps([{
    "txid": tx['txid'],
    "vout": tx['vout'][0]['n']
}], separators=(',', ':'))

amount = tx['vout'][0]['value'] - 0.01

tx_out = json.dumps([{
    bcli("getnewaddress"): amount
}], separators=(',', ':'))

raw_tx = bcli(f'createrawtransaction {tx_in} {tx_out}')

In [24]:
# Sign transaction
signed_tx = json.loads(bcli(f'signrawtransactionwithwallet {raw_tx}'))['hex']
signed_tx

'0200000000010106abca1dc71e91ded34e1b3a601e2115a515a2c9a3505c1bf6abd79ef184e1720000000000fdffffff01c0b6f3940000000016001493d50a6d6677ee2840657ea867b63128f3b27b370247304402201f2aa0504fbe41cceac2db7306405bfde9f744268f6bbeb249b6d34609eb72700220760490a762e6bc9f390093deb0c2b686c212ee29043f4c54861629cc8c877e4701210288a4588d9cf866dcc8a8b21910f97846fa0f43c7fa754c6d4820a8673dc1083400000000'

In [25]:
# Check if is valid txW
json.loads(bcli(f'testmempoolaccept ["{signed_tx}"]'))[0]

{'txid': '01662551f06debf625c73eda455ae33587a0254a9fd0ed156a987b40c434c526',
 'wtxid': '551e47354bee833ccd34ce0a6bc4109c16e3ffe3b28a2058b6098ba8dacc4203',
 'allowed': True,
 'vsize': 110,
 'fees': {'base': 0.01,
  'effective-feerate': 0.09090909,
  'effective-includes': ['551e47354bee833ccd34ce0a6bc4109c16e3ffe3b28a2058b6098ba8dacc4203']}}

In [26]:
# Decode transaction
decoded_tx = json.loads(bcli(f'decoderawtransaction {raw_tx}'))
decoded_tx

{'txid': '01662551f06debf625c73eda455ae33587a0254a9fd0ed156a987b40c434c526',
 'hash': '01662551f06debf625c73eda455ae33587a0254a9fd0ed156a987b40c434c526',
 'version': 2,
 'size': 82,
 'vsize': 82,
 'weight': 328,
 'locktime': 0,
 'vin': [{'txid': '72e184f19ed7abf61b5c50a3c9a215a515211e603a1b4ed3de911ec71dcaab06',
   'vout': 0,
   'scriptSig': {'asm': '', 'hex': ''},
   'sequence': 4294967293}],
 'vout': [{'value': 24.99,
   'n': 0,
   'scriptPubKey': {'asm': '0 93d50a6d6677ee2840657ea867b63128f3b27b37',
    'desc': 'addr(bcrt1qj02s5mtxwlhzssr9065x0d339remy7ehlv5rwn)#6v43xdht',
    'hex': '001493d50a6d6677ee2840657ea867b63128f3b27b37',
    'address': 'bcrt1qj02s5mtxwlhzssr9065x0d339remy7ehlv5rwn',
    'type': 'witness_v0_keyhash'}}]}

In [27]:
decoded_tx['version']
decoded_tx['size']

82

In [28]:
decoded_tx

{'txid': '01662551f06debf625c73eda455ae33587a0254a9fd0ed156a987b40c434c526',
 'hash': '01662551f06debf625c73eda455ae33587a0254a9fd0ed156a987b40c434c526',
 'version': 2,
 'size': 82,
 'vsize': 82,
 'weight': 328,
 'locktime': 0,
 'vin': [{'txid': '72e184f19ed7abf61b5c50a3c9a215a515211e603a1b4ed3de911ec71dcaab06',
   'vout': 0,
   'scriptSig': {'asm': '', 'hex': ''},
   'sequence': 4294967293}],
 'vout': [{'value': 24.99,
   'n': 0,
   'scriptPubKey': {'asm': '0 93d50a6d6677ee2840657ea867b63128f3b27b37',
    'desc': 'addr(bcrt1qj02s5mtxwlhzssr9065x0d339remy7ehlv5rwn)#6v43xdht',
    'hex': '001493d50a6d6677ee2840657ea867b63128f3b27b37',
    'address': 'bcrt1qj02s5mtxwlhzssr9065x0d339remy7ehlv5rwn',
    'type': 'witness_v0_keyhash'}}]}

In [46]:
def split_transaction(txns):
    offset = 0

    version = txns[offset:offset+4]
    offset += 4

    marker = txns[offset:offset+1]
    offset += 1

    flag = txns[offset:offset+1]
    offset += 1
    
    tx_in_count = txns[offset:offset+1]
    offset += 1

    tx_in = []
    for _ in range(int.from_bytes(tx_in_count)):
        previous_output = txns[offset:offset+36]
        offset += 36

        script_length = txns[offset:offset+1]
        offset += 1

        signature_script = txns[offset:offset+int.from_bytes(script_length)]
        offset += int.from_bytes(script_length)

        sequence = txns[offset:offset+4]
        offset += 4

        tx_in.append({
            "previous_output": previous_output.hex(),
            "script_length": script_length.hex(),
            "signature_script": signature_script.hex(),
            "sequence": sequence.hex()
        })

    tx_out_count = txns[offset:offset+1]
    offset += 1

    tx_out = []
    for _ in range(int.from_bytes(tx_out_count)):
        value = txns[offset:offset+8]
        offset += 8

        pk_script_length = txns[offset:offset+1]
        offset += 1

        pk_script = txns[offset:offset+int.from_bytes(pk_script_length)]
        offset += int.from_bytes(pk_script_length)

        tx_out.append({
            "value": value.hex(),
            "pk_script_length": pk_script_length.hex(),
            "pk_script": pk_script.hex()
        })

    witness_count = txns[offset:offset+1]
    offset += 1

    witness = []
    for _ in range(int.from_bytes(witness_count)):
        size = txns[offset:offset+1]
        offset += 1

        item = txns[offset:offset+int.from_bytes(size)]
        offset += int.from_bytes(size)

        witness.append({
            "size": size.hex(),
            "item": item.hex()
        })
    
    lock_time = txns[offset:offset+4]
    offset += 4
    
    return {
        "version": version.hex(),
        "marker": marker.hex(),
        "flag": flag.hex(),
        "tx_in_count": tx_in_count.hex(),
        "tx_in": tx_in,
        "tx_out_count": tx_out_count.hex(),
        "tx_out": tx_out,
        "witness": witness,
        "lock_time": lock_time.hex()
    } 

In [47]:
split_transaction(bytes.fromhex("0200000000010106abca1dc71e91ded34e1b3a601e2115a515a2c9a3505c1bf6abd79ef184e1720000000000fdffffff01c0b6f3940000000016001493d50a6d6677ee2840657ea867b63128f3b27b370247304402201f2aa0504fbe41cceac2db7306405bfde9f744268f6bbeb249b6d34609eb72700220760490a762e6bc9f390093deb0c2b686c212ee29043f4c54861629cc8c877e4701210288a4588d9cf866dcc8a8b21910f97846fa0f43c7fa754c6d4820a8673dc1083400000000"))

{'version': '02000000',
 'marker': '00',
 'flag': '01',
 'tx_in_count': '01',
 'tx_in': [{'previous_output': '06abca1dc71e91ded34e1b3a601e2115a515a2c9a3505c1bf6abd79ef184e17200000000',
   'script_length': '00',
   'signature_script': '',
   'sequence': 'fdffffff'}],
 'tx_out_count': '01',
 'tx_out': [{'value': 'c0b6f39400000000',
   'pk_script_length': '16',
   'pk_script': '001493d50a6d6677ee2840657ea867b63128f3b27b37'}],
 'witness': [{'size': '47',
   'item': '304402201f2aa0504fbe41cceac2db7306405bfde9f744268f6bbeb249b6d34609eb72700220760490a762e6bc9f390093deb0c2b686c212ee29043f4c54861629cc8c877e4701'},
  {'size': '21',
   'item': '0288a4588d9cf866dcc8a8b21910f97846fa0f43c7fa754c6d4820a8673dc10834'}],
 'lock_time': '00000000'}

In [ ]:
def split_block_transactions(count: int, txns: bytes) -> list:
    offset = 0
    transactions = []

    for _ in range(count):
        version = txns[offset:offset+4]
        offset += 4
        
        tx_in_count = txns[offset:offset+1]
        offset += 1

        tx_in = bytes()
        for _ in range(int.from_bytes(tx_in_count)):
            previous_output = txns[offset:offset+36]
            offset += 36

            script_length = txns[offset:offset+1]
            offset += 1

            signature_script = txns[offset:offset+int.from_bytes(script_length)]
            offset += int.from_bytes(script_length)

            sequence = txns[offset:offset+4]
            offset += 4

            tx_in += previous_output + script_length + signature_script + sequence

        tx_out_count = txns[offset:offset+1]
        offset += 1

        tx_out = bytes()
        for _ in range(int.from_bytes(tx_out_count)):
            value = txns[offset:offset+8]
            offset += 8

            pk_script_length = txns[offset:offset+1]
            offset += 1

            pk_script = txns[offset:offset+int.from_bytes(pk_script_length)]
            offset += int.from_bytes(pk_script_length)

            tx_out += value + pk_script_length + pk_script
        
        lock_time = txns[offset:offset+4]
        offset += 4

        transactions.append(version + tx_in_count + tx_in + tx_out_count + tx_out + lock_time)

    return transactions